# Chapter 7. Hypothesis and inference

## 7.1 Statistical hypothesis testing

## 7.2 Example: flipping a coin

Imagine we have a coin and we want to test whether it’s fair. We’ll make the assumption that the coin has some probability p of landing heads, and so our null hypothesis is that the coin is fair—that is, that p = 0.5. We’ll test this against the alternative hypothesis p ≠ 0.5.

In particular, our test will involve flipping the coin some number, n, times and counting the number of heads, X. Each coin flip is a Bernoulli trial, which means that X is a Binomial(n,p) random variable, which we can approximate using the normal distribution

In [1]:
from typing import Tuple
import math 

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    '''Returns mu and sigma corresponding to a Binomial(n, p)'''
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [2]:
def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

# the normal cdf is the probability the variable is below a threshold
normal_probability_below = normal_cdf

# It's above the threshold if it is not below the threshold
def normal_probability_above(lo: float,
                             mu: float = 0,
                             sigma: float = 1) -> float:
    '''The probability that an N(mu, sigma) is greater than lo'''
    return 1 - normal_cdf(lo, mu, sigma)

# It is between if it is less than hi, but not less than lo
def normal_probability_between(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float: 
    '''The probability that an N(mu, sigma) is between lo and hi'''
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# It is outside if it is not between
def normal_probability_outside(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    '''The probability that an N(mu, sigma) is not between lo and hi'''
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [3]:
def inverse_normal_cdf(p: float,
                       mu: float = 0,
                       sigma: float = 1,
                       tolerance: float = 0.00001) -> float:
    """Find approximate inverse using binary search"""

    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)

    low_z = -10.0                      # normal_cdf(-10) is (very close to) 0
    hi_z  =  10.0                      # normal_cdf(10)  is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # Consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            low_z = mid_z              # Midpoint too low, search above it
        else:
            hi_z = mid_z               # Midpoint too high, search below it

    return mid_z

def normal_upper_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    '''Returns the z for which P(Z <= z) = probability'''
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    '''Returns the z for which P(Z >= z) = probability'''
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                            mu: float = 0,
                            sigma: float = 1) -> Tuple[float, float]:
    '''Returns the symmetric (about the mean) bounds that contain the specified probability'''
    tail_probability = (1 - probability) / 2
    
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [4]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [5]:
print(mu_0, sigma_0)

500.0 15.811388300841896


In [6]:
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [7]:
print(lower_bound, upper_bound)

469.01026640487555 530.9897335951244


In [8]:
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [9]:
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

In [10]:
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
power

0.8865480012953671

In [11]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
power

0.9363794803307173

## 7.3 p-values

In [25]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    '''How likely are we to see a value at least as extreme as x (in either direction) if our values are from an N(mu, sigma)'''
    if x >= mu:
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        return 2 * normal_probability_below(x, mu, sigma)

two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

In [23]:
import random 

extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0 # count # of heads
                    for _ in range (1000)) # in 1000 flips 
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1 # count how often the # is extreme

assert 59 < extreme_value_count < 65, f'{extreme_value_count}'

AssertionError: 58

In [26]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

In [27]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [30]:
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582072

In [31]:
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

**Note:** Make sure your data is roughly normally distributed before using normal_probability_above to compute p-values. The annals of bad data science are filled with examples of people opining that the chance of some observed event occurring at random is one in a million, when what they really mean is “the chance, assuming the data is distributed normally,” which is fairly meaningless if the data isn’t. There are various statistical tests for normality, but even plotting the data is a good start.

## 7.4 Confidence intervals

In [33]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

In [34]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

In [35]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
normal_two_sided_bounds(0.95, mu, sigma)

(0.5091095927295919, 0.5708904072704082)

## 7.5 p-hacking

In [36]:
from typing import List

def run_experiment() -> List[bool]:
    '''Filps a fair coin 1000 times, True = heads, False = tails'''
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    '''Using the 5% significance levels'''
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment 
                      for experiment in experiments
                      if reject_fairness(experiment)])

assert num_rejections == 46

What this means is that if you’re setting out to find “significant” results, you usually can. Test enough hypotheses against your dataset, and one of them will almost certainly appear significant.

## 7.6 Example: running an a/b test

In [37]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N 
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

In [38]:
def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [39]:
z = a_b_test_statistic(1000, 200, 1000, 180)
z

-1.1403464899034472

In [40]:
two_sided_p_value(z)

0.254141976542236

In [41]:
z = a_b_test_statistic(1000, 200, 1000, 150)
z

-2.948839123097944

In [42]:
two_sided_p_value(z)

0.003189699706216853

## 7.7 Bayesian inference

In [43]:
def B(alpha: float, beta: float) -> float:
    '''A normalizing constant so that the total probability is 1'''
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

## 7.8 For further exploration